In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split,KFold, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector

In [2]:
from ISLP import load_data
Credit = load_data('Credit')

In [3]:
Credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         400 non-null    int64   
 1   Income     400 non-null    float64 
 2   Limit      400 non-null    int64   
 3   Rating     400 non-null    int64   
 4   Cards      400 non-null    int64   
 5   Age        400 non-null    int64   
 6   Education  400 non-null    int64   
 7   Gender     400 non-null    category
 8   Student    400 non-null    category
 9   Married    400 non-null    category
 10  Ethnicity  400 non-null    category
 11  Balance    400 non-null    int64   
dtypes: category(4), float64(1), int64(7)
memory usage: 27.2 KB


In [4]:
X = Credit.drop(['Balance','ID'], axis=1)
y = Credit['Balance']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=24)

In [5]:
ohe = OneHotEncoder(handle_unknown='ignore')
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude='category')  ),
                             (ohe, make_column_selector(dtype_include='category') ),
                             verbose_feature_names_out=False)
dtr = DecisionTreeRegressor(random_state=24)
pipe_dtr = Pipeline([('CT',ct),('DT',dtr)])
knn = KNeighborsRegressor()
pipe_knn = Pipeline([('CT',ct),('KNN',knn)])
el = ElasticNet()
pipe_el = Pipeline([('CT',ct),('EL',el)])

In [6]:
voting = VotingRegressor([('DT',pipe_dtr),('KNN',pipe_knn),('EL',pipe_el)])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print(r2_score(y_test, y_pred))

0.914557198194179


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


In [7]:
pipe_dtr.fit(X_train, y_train)
y_pred = pipe_dtr.predict(X_test)
print(r2_score(y_test, y_pred))

0.899162982269506


In [8]:
pipe_knn.fit(X_train, y_train)
y_pred = pipe_knn.predict(X_test)
print(r2_score(y_test, y_pred))

0.7899310404076199


In [9]:
pipe_el.fit(X_train, y_train)
y_pred = pipe_el.predict(X_test)
print(r2_score(y_test, y_pred))

0.9057359592636076


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


In [10]:
voting = VotingRegressor([('DT',pipe_dtr),('KNN',pipe_knn),('EL',pipe_el)],
                         weights=[5,3,5])
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print(r2_score(y_test, y_pred))

0.9232798673922049


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


OR

In [12]:
ohe = OneHotEncoder(handle_unknown='ignore')
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude='category')  ),
                             (ohe, make_column_selector(dtype_include='category') ),
                             verbose_feature_names_out=False)
dtr = DecisionTreeRegressor(random_state=24)
knn = KNeighborsRegressor()
el = ElasticNet()
voting = VotingRegressor([('DT',dtr),('KNN',knn),('EL',el)], 
                          weights=[6,3,6])
pipe_vt = Pipeline([('CT',ct),('VOT',voting)])
pipe_vt.fit(X_train, y_train)
y_pred = pipe_vt.predict(X_test)
print(r2_score(y_test, y_pred))

C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.054e+06, tolerance: 6.154e+03
  model = cd_fast.enet_coordinate_descent(


0.9253532376876176


Grid Search 

In [14]:
pipe_vt.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000027528F356A0>),
                                   ('onehotencoder',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000027528F36390>)],
                     verbose_feature_names_out=False)),
  ('VOT',
   VotingRegressor(estimators=[('DT', DecisionTreeRegressor(random_state=24)),
                               ('KNN', KNeighborsRegressor()),
                               ('EL', ElasticNet())],
                   weights=[6, 3, 6]))],
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000027528F356A0>),
 

In [15]:
kfold = KFold(n_splits=5, shuffle=True, random_state=24)
params = {'VOT__KNN__n_neighbors':[2,5,7],
          'VOT__EL__alpha':np.linspace(0.001, 3, 3),
          'VOT__DT__max_depth':[None, 3, 5],
          'VOT__DT__min_samples_leaf':[1,5,10],
          'VOT__DT__min_samples_split':[2,5,10],
          'VOT__weights':[[6,3,6],[7,4,7],[9,2,9]]}
gcv = GridSearchCV(pipe_vt, param_grid=params, cv=kfold, verbose=3)
gcv.fit(X, y)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.564e+06, tolerance: 6.884e+03
  model = cd_fast.enet_coordinate_descent(


[CV 1/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.947 total time=   0.0s


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.459e+06, tolerance: 6.842e+03
  model = cd_fast.enet_coordinate_descent(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.580e+06, tolerance: 6.921e+03
  model = cd_fast.enet_coordinate_descent(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.510e+06, tolerance: 6.558e+03
  

[CV 2/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.870 total time=   0.0s
[CV 3/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.926 total time=   0.0s
[CV 4/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.949 total time=   0.0s


...

[CV 1/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=5, VOT__weights=[7, 4, 7];, score=0.901 total time=   0.0s
[CV 2/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=5, VOT__weights=[7, 4, 7];, score=0.850 total time=   0.0s
[CV 3/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=5, VOT__weights=[7, 4, 7];, score=0.840 total time=   0.0s
[CV 4/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=5, VOT__weights=[7, 4, 7];, score=0.887 total time=   0.0s
[CV 5/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=5, VOT__weights=[7, 4, 7];, score=0.896 total time=   0.0s
[CV 1/5] END VO

C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.091e+06, tolerance: 6.558e+03
  model = cd_fast.enet_coordinate_descent(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.360e+06, tolerance: 6.511e+03
  model = cd_fast.enet_coordinate_descent(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.716e+06, tolerance: 6.884e+03
  

[CV 4/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=7, VOT__weights=[6, 3, 6];, score=0.889 total time=   0.0s
[CV 5/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=7, VOT__weights=[6, 3, 6];, score=0.899 total time=   0.0s
[CV 1/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=7, VOT__weights=[7, 4, 7];, score=0.904 total time=   0.0s
[CV 2/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=7, VOT__weights=[7, 4, 7];, score=0.846 total time=   0.0s
[CV 3/5] END VOT__DT__max_depth=5, VOT__DT__min_samples_leaf=10, VOT__DT__min_samples_split=10, VOT__EL__alpha=3.0, VOT__KNN__n_neighbors=7, VOT__weights=[7, 4, 7];, score=0.842 total time=   0.0s
[CV 4/5] END VO

C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.902e+06, tolerance: 8.434e+03
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000027528F356A0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_column_sel...
                                                                     DecisionTreeRegressor(random_state=24)),
                                                                    ('KNN',
                                                                     KNeighborsRegressor()),
                                                                    ('EL',
                                                                     ElasticNet())],
                                                        weights=[6, 3, 6]))]),
             param_grid={'VOT__DT__max_depth': [None, 3, 5],
                         'VOT__DT__min_samples_leaf': [1, 5, 10],
                         'VOT__DT__min_samples_split': [2, 5, 10],
                         'VOT__EL__alpha': array([1.0000e-03, 1.5005e+00, 3.0000e+00]),
                         'VOT__KNN__n_neighbors': [2, 5, 7],
                         'VOT__weights': [[6, 3, 6], [7, 4, 7], [9, 2, 9]]},
             verbose=3)

In [16]:
print(gcv.best_params_)
print(gcv.best_score_)

{'VOT__DT__max_depth': None, 'VOT__DT__min_samples_leaf': 1, 'VOT__DT__min_samples_split': 5, 'VOT__EL__alpha': 0.001, 'VOT__KNN__n_neighbors': 2, 'VOT__weights': [9, 2, 9]}
0.9413326334165933


In [17]:
pd_cv = pd.DataFrame(gcv.cv_results_)
pd_cv.shape

(729, 19)

In [18]:
pd_cv.to_csv("C:/Python/DAI_Oct24/gridresults.csv", index=False)